<span style="color:#a7d129; font-size:30px;">Description</span>  

This notebook use model gemini-2.5-flash from google ai studio. The language used is English

<span style="color:#a7d129; font-size:30px;">Library</span>  

In [1]:
from dotenv import load_dotenv
from pathlib import Path

import google.genai as genai
import time
import json
import os

In [ ]:
# Custom function
import sys
sys.path.append("../src")

from my_db_manager import GameHistory

<span style="color:#a7d129; font-size:30px;">Initiation</span>  

In [3]:
# Model configuration
model = "gemini-2.5-flash"
api_key = "API_KEY_FOR_001"

In [4]:
# Path
root_path = Path.cwd().parent

env_path = root_path / "config" / ".env"
lore_path = root_path / "database" / "world" / "world_a.json"
db_path = root_path / "database" / "history.db"

In [5]:
# Add costume code (database manager)
gh = GameHistory(db_path = db_path) 

In [6]:
# Import and initiation lore game (.json)
with open(lore_path, "r", encoding="utf-8") as f:
    lore = json.load(f)

<span style="color:#a7d129; font-size:30px;">Load</span>  

In [7]:
# Load API key from .env
load_dotenv(dotenv_path=env_path)

api_key = os.getenv(api_key)

In [8]:
# Configuration
client = genai.Client(api_key=api_key)

In [9]:
parent_loc_list = list(lore["location"].keys())

loc_list = [ 
    sub_keys
    for parent in parent_loc_list
    for sub_keys in lore["location"][parent].keys() 
    ]

all_loc_list = parent_loc_list + loc_list

<span style="color:#a7d129; font-size:30px;">Create database for history</span>  

In [10]:
# Configuration
gh.create(tab_name = "history_001")

<span style="color:#a7d129; font-size:30px;">Play</span>  

In [11]:
# Get last history for context to model
def get_history(speaker_code, intro_text, limit=5):
    history = {
        "narrative" : intro_text,
        "all_history": {},
        "one_person_history": {}
    }

    # Enter all history
    for code, message, timestamp in gh.get(limit=limit):
        history["all_history"][timestamp] = f"{code}: {message}"

    # Enter history one speaker
    for code, message, timestamp in gh.get_oph(speaker_code, limit=limit):
        history["one_person_history"][timestamp] = f"{code}: {message}"

    return history

In [12]:
# Generate prompt
def gen_prompt(inp, char, parent_loc, keys, intro_text):
    # Copy data so as not to change the original
    lore_copy = lore["character"][char].copy()
    loc_copy = {"location":{parent_loc:{**keys}}}
    history_copy = get_history(char, intro_text)
    inp_copy = {"question": inp}

    # Merge all dict
    final_prompt = {**lore_copy, **loc_copy, **history_copy, **inp_copy}

    # Convert it to parts format
    parts = [{"text": f"{key.capitalize()}: {value}"} for key, value in final_prompt.items()]

    return [
        {
            "role": "user",
            "parts": parts
        }
    ]

In [13]:
def format_elapsed(start_time, end_time):
    elapsed = end_time - start_time
    minutes, sec = divmod(int(elapsed), 60)
    milliseconds = int((elapsed - int(elapsed)) * 1000)
    return f"{minutes:02d}m:{sec:02d}s:{milliseconds:03d}ms"

In [ ]:
def request(inp:str, char, parent_loc, keys, intro_text):
    print(f"You                          : {inp}")
    
    prompt = gen_prompt(inp, char, parent_loc, keys, intro_text)
        
    start_time = time.time()

    # Send a prompt to the model
    response = client.models.generate_content(
        model=model,   
        contents=prompt
    )

    end_time = time.time()

    timestamp = format_elapsed(start_time, end_time)

    real_response = response.text
    print(f"{char}[{timestamp}] : {real_response}")

    return 

In [15]:
def multi_loc(loc, text):
    keys = {"sub_loc" : list(lore["location"][loc].keys())}
    return text + f"{loc} ?", keys

In [16]:
def escape(inp):
    if inp.lower() in ["/exit", "/quit"]:
        print("Exiting chat...")
        return True  
    return False     

In [17]:
def get_loc(inp: str, current_loc, current_keys):
    parts = inp.strip("/").split("/") if inp.strip("/") else []
    
    if not parts or parts[0] not in all_loc_list:
        print("Invalid location!!")
        return current_loc, current_keys, False
    
    text = f"Are you sure you wanna leave {current_loc}, and go to "

    if len(parts) == 1 :
        final_text, keys = multi_loc(parts[0], text)
    else :
        parent_loc, loc = parts[0], parts[1] 
        if loc not in loc_list:
            print("Invalid location!!")
            return current_loc, current_keys, False
        keys = lore["location"][parent_loc][loc].copy()
        final_text = text + f"{loc} in {parent_loc} ?"

    inp_confirm = input(f"{final_text} (yes/no): ").strip().lower()

    if inp_confirm in ["no","n"]:
        return current_loc, current_keys, False
    elif inp_confirm in ["yes", "y"]:
        return parts[0], keys, True


In [18]:
# Start simulation chat
def chat(char: str = "consciousness", parent_loc:str = "station", keys=None):
    if keys is None:
        keys = lore["location"]["station"]["platform"].copy()

    stop_chat = False
    cond = "Where is my wallet?"
    inp = ""
    current_loc = parent_loc

    intro_text = "On the train platform, a man stood looking lost (You). He patted his front and back pockets, " \
    "\nunzipped his jacket, and checked the inside pocket. As he reached deep inside, a station officer pulled him back. " \
    "\nIn the next heartbeat, the roar of a passing train filled his ears. He gave a small bow of thanks, nodded."

    print(intro_text)
    
    # Make sure the initial question is appropriate
    # while inp.lower() != cond.lower():
    #     inp = input(f'You : (At first type, please type "{cond}") ')
        # if escape(inp):
        #         break
    
    # if stop_chat:
    #     return
    
    # request(inp, char, parent_loc, keys, intro_text)

    # After the initial question, enter the main loop.
    while True:
        inp = input("You    : ")
        if not inp :
            continue 

        if inp.startswith("/"):
            if escape(inp):
                break

            new_loc, new_keys, moved = get_loc(inp.lower(), current_loc, keys)

            if not moved:
                print(f"We are not moving")
                continue
            else:
                current_loc, keys = new_loc, new_keys 
                print(f"Current location is {current_loc}")
            continue
            
        request(inp, char, current_loc, keys, intro_text)
        # add_history("P", "Apa yang kamu lihat?")


In [19]:
chat()

On the train platform, a man stood looking lost (You). He patted his front and back pockets, 
unzipped his jacket, and checked the inside pocket. As he reached deep inside, a station officer pulled him back. 
In the next heartbeat, the roar of a passing train filled his ears. He gave a small bow of thanks, nodded.
You                         : hello
consciousness:04s:957ms] : Lost... *thump*... The wallet.
Exiting chat...
